### Create daily input files (from the hourly files)

In [ ]:
proj_dir="/path/to/main_project_folder/" # edit this line

import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nc4
import re
import sys
import os
import time
import multiprocessing as mp

years = range(1979, 2022)

region_list = ['northcentral_north_america', 
               'southcentral_north_america', 
               'southeastern_north_america', 
               'southwestern_europe', 
               'western_europe', 
               'central_europe', 
               'eastern_europe', 
               'northeastern_europe', 
               'northeastern_asia', 
               'southeastern_asia', 
               'northsouthern_south_america', 
               'southsouthern_south_america', 
               'southwestern_africa', 
               'southeastern_africa', 
               'southwestern_australia', 
               'southeastern_australia', 
               'west_texas',
               'east_texas',
              ]

In [ ]:
def parallel_by_yr_daily_max(yr):
    path_in = proj_dir+"/input_data_ERA5/"+region_str+"/hourly/"+var_in+"/"
    path_out = proj_dir+"/input_data_ERA5/"+region_str+"/daily/"+var_out+"/"
    
    f_in = '_'.join([region_str, var_in, 'hourly', str(yr)+'.nc']) 
    f_out = '_'.join([region_str, var_out, 'daily', str(yr)+'.nc']) 

    if os.path.exists(path_in + f_in):
        if not os.path.exists(path_out + f_out):
            ds = xr.open_dataset(path_in + f_in)

            daily_var = ds.resample(time='24H').max(dim="time", skipna=True, keep_attrs=True)
            daily_var = daily_var.rename({var_in: var_out})
            daily_var.to_netcdf(path_out + f_out)

def parallel_by_yr_daily_mean(yr):
    path_in = proj_dir+"/input_data_ERA5/"+region_str+"/hourly/"+var_in+"/"
    path_out = proj_dir+"/input_data_ERA5/"+region_str+"/daily/"+var_in+"/"

    f_in = '_'.join([region_str, var_in, 'hourly', str(yr)+'.nc']) 
    f_out = '_'.join([region_str, var_out, 'daily', str(yr)+'.nc']) 

    if os.path.exists(path_in + f_in):
        if not os.path.exists(path_out + f_out):
            ds = xr.open_dataset(path_in + f_in)

            daily_var = ds.resample(time='24H').mean(dim="time", skipna=True, keep_attrs=True)
            daily_var = daily_var.rename({var_in: var_out})
            daily_var.to_netcdf(path_out + f_out)


In [ ]:
############ calculate daily MEAN for swl1 #############
global var_in
var_in="swvl1"
global var_out
var_out = "swvl1"

for region_str_loc in region_list:
    global region_str
    region_str = region_str_loc
    print(region_str)
    with mp.Pool(40) as p:
        p.map(parallel_by_yr_daily_mean, years)

In [ ]:
############ calculate daily MEAN for geopotential #############
global var_in
var_in="z"
global var_out
var_out = "z"

for region_str_loc in region_list:
    global region_str
    region_str = region_str_loc
    print(region_str)
    with mp.Pool(40) as p:
        p.map(parallel_by_yr_daily_mean, years)

In [ ]:
############ calculate daily MAX for t2m #############
global var_in
var_in="t2m"
global var_out
var_out = "tmax"

for region_str_loc in region_list:
    global region_str
    region_str = region_str_loc
    print(region_str)
    with mp.Pool(40) as p:
        p.map(parallel_by_yr_daily_max, years)